## Explore NV Bluefield DPU

author : yiakwy

In [4]:
!hostnamectl
!cat /etc/os-release
!lspci | grep -i BlueField

 Static hostname: doca01-a
       Icon name: computer-vm
         Chassis: vm
      Machine ID: 3e41ee15ec634257bb248d9b7a563724
         Boot ID: 6eb0bedf559b4207b3468d0c676cf247
  Virtualization: vmware
Operating System: ]8;;https://www.ubuntu.com/Ubuntu 22.04.3 LTS]8;;              
          Kernel: Linux 5.15.0-86-generic
    Architecture: x86-64
 Hardware Vendor: VMware, Inc.
  Hardware Model: VMware7,1
PRETTY_NAME="Ubuntu 22.04.3 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.3 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy
0b:00.0 Ethernet controller: Mellanox Technologies MT42822 BlueField-2 integrated ConnectX-6 Dx network controller (rev 01)
0b:00.1 Ethernet controller: Mellanox Technologies MT42822 BlueField-2 in

In [5]:
# check Host info, we have 2 host connected to 2 DPU
!lscpu

Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         43 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  8
  On-line CPU(s) list:   0-7
Vendor ID:               GenuineIntel
  Model name:            Intel(R) Xeon(R) CPU E5-2690 v4@ 2.60GHz
    CPU family:          6
    Model:               79
    Thread(s) per core:  1
    Core(s) per socket:  8
    Socket(s):           1
    Stepping:            1
    BogoMIPS:            5199.99
    Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mc
                         a cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscal
                         l nx pdpe1gb rdtscp lm constant_tsc arch_perfmon nopl x
                         topology tsc_reliable nonstop_tsc cpuid tsc_known_freq 
                         pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic 
                         movbe popcnt tsc_deadline_timer aes xsave av

In [12]:
# download sdk manager https://developer.nvidia.com/sdk-manager
!which sdkmanager
!sdkmanager --ver

# install doca software with passwd
# !sudo bfb-install --rshim rshim0 --bfb ./DOCA_2.2.0_BSP_4.2.0_Ubuntu_22.04-2.23-07.prod.bfb --config bf.cfg

/usr/bin/sdkmanager
2.0.0.11405


In [2]:
# ssh-copy-id 'ubuntu@192.168.101.11'
# check dpu machine
DPU='ubuntu@192.168.101.11'

In [9]:
!ssh -oStrictHostKeyChecking=no {DPU} "df -lh && ls -lh && sudo bfvcheck"

Filesystem      Size  Used Avail Use% Mounted on
tmpfs           1.6G  4.3M  1.6G   1% /run
/dev/mmcblk0p2   59G  5.9G   50G  11% /
tmpfs           7.8G  4.3M  7.8G   1% /dev/shm
tmpfs           5.0M  4.0K  5.0M   1% /run/lock
/dev/mmcblk0p1   50M  8.6M   41M  18% /boot/efi
shm              64M     0   64M   0% /run/containerd/io.containerd.grpc.v1.cri/sandboxes/3ea0fc53127109a97fa2f067a6619afcfb10996782a3917ad95fef4c3bb98226/shm
tmpfs           1.6G     0  1.6G   0% /run/user/1000
total 0
Beginning version check...

-RECOMMENDED VERSIONS-
ATF: v2.2(release):4.2.0-80-g0ad5a0e
UEFI: 4.2.0-56-ge28fcb7
FW: 24.38.1002

-INSTALLED VERSIONS-
ATF: v2.2(release):4.2.0-80-g0ad5a0e
UEFI: 4.2.0-56-ge28fcb7
FW: 24.38.1002

Version check complete.
No issues found.


In [12]:
# note you can download very old dpu samples on gitlab, but they don't offer compilation files
# DPU uses meson to build c files. Meson is somewhat like CMake to generate ninja build files (by default) or make Makefiles
!scp -r {DPU}:/opt/mellanox/doca/samples ~/dpu-v1.5.1-samples
# dont' change the name, since dpu-v1.5.1-samples uses this for compilation
!scp -r {DPU}:/opt/mellanox/doca/applications ~/applications

rdma_common.h                                 100% 4858     3.0MB/s   00:00    
rdma_write_responder_main.c                   100% 2579     2.2MB/s   00:00    
rdma_write_responder_sample.c                 100% 4903     3.3MB/s   00:00    
meson.build                                   100% 1523     1.3MB/s   00:00    
rdma_write_immediate_responder_sample.c       100% 6578     4.7MB/s   00:00    
meson.build                                   100% 1533     1.3MB/s   00:00    
rdma_write_immediate_responder_main.c         100% 2629     2.2MB/s   00:00    
rdma_common.c                                 100%   28KB   6.3MB/s   00:00    
meson.build                                   100% 1533     1.1MB/s   00:00    
rdma_write_immediate_requester_sample.c       100% 9567     7.8MB/s   00:00    
rdma_write_immediate_requester_main.c         100% 2672     1.9MB/s   00:00    
meson.build                                   100% 1522     1.3MB/s   00:00    
rdma_read_responder_main.c              

In [17]:
# dpu-v1.5.1-samples uses this path "/opt/mellanox/doca/applications" for compilation
!echo @doca | sudo -S ln -s ./applications /opt/mellanox/doca/applications
!ls -lh /opt/mellanox/doca/

[sudo] password for doca: ln: failed to create symbolic link '/opt/mellanox/doca/applications': File exists
total 12K
lrwxrwxrwx 1 root root   14 Jan  5 08:48 applications -> ./applications
drwxr-xr-x 3 root root 4.0K Oct 11 02:34 infrastructure
drwxr-xr-x 3 root root 4.0K Oct 11 02:34 lib
drwxr-xr-x 2 root root 4.0K Oct 11 02:35 tools


In [3]:
!ssh -oStrictHostKeyChecking=no {DPU} "/opt/mellanox/iproute2/sbin/mlxdevm port"

pci/0000:03:00.0/294944: type eth netdev en3f0pf0sf0 flavour pcisf controller 0 pfnum 0 sfnum 0
  function:
    hw_addr 02:cf:b1:af:76:ee state active opstate attached roce true max_uc_macs 128 trust off
pci/0000:03:00.1/360480: type eth netdev en3f1pf1sf0 flavour pcisf controller 0 pfnum 1 sfnum 0
  function:
    hw_addr 02:98:9f:bc:2f:9d state active opstate attached roce true max_uc_macs 128 trust off
